# Payoff Binary barrier

Payout fixe si barriere franchie (BS simplifie).


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_barrier
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
strike_ref = spot_ref
bar_ref = spot_ref*1.1


## Payoff interactif
Barriere, knock, payout fixe, sigma, r, T.


In [ ]:

sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.6, max=spot_ref*1.4, step=1.0, description='S0')
bar_slider = widgets.FloatSlider(value=bar_ref, min=spot_ref*0.5, max=spot_ref*1.8, step=1.0, description='Barriere')
direction_dd = widgets.Dropdown(options=[('Up','up'),('Down','down')], value='up', description='Direction')
knock_dd = widgets.Dropdown(options=[('Out','out'),('In','in')], value='out', description='Knock')
payout_slider = widgets.FloatSlider(value=1.0, min=0.5, max=5.0, step=0.5, description='Payout')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        s0 = spot_slider.value
        bar = bar_slider.value
        view_dyn = view_barrier(s0, strike_ref, bar, direction=direction_dd.value, knock=knock_dd.value, binary=True, payout=payout_slider.value, r=r_slider.value, q=0.0, sigma=sigma_slider.value, T=T_slider.value)
        premium = float(view_dyn.get('premium', 0.0))
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        payoff_s0 = float(np.interp(s0, s_grid, payoff_grid))
        pnl_s0 = payoff_s0 - premium
        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff binaire')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axvline(bar, color='firebrick', linestyle=':', label='Barriere = {:.2f}'.format(bar))
        ax.axvline(s0, color='crimson', linestyle='-.', label='S0 = {:.2f}'.format(s0))
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Binary barrier')
        plt.show()
        plt.close(fig)
        display(Markdown('Prime ~ {:.4f}

- Payoff @ S0 = {:.4f}
- P&L net = {:.4f}'.format(premium, payoff_s0, pnl_s0)))

for sl in (spot_slider, bar_slider, direction_dd, knock_dd, payout_slider, sigma_slider, r_slider, T_slider):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([spot_slider, bar_slider, direction_dd, knock_dd, payout_slider, sigma_slider, r_slider, T_slider, output]))
